<a href="https://colab.research.google.com/github/Leads-DigiSaka-System/strawburn_alerts/blob/main/SunogDayame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 0) Install & imports
!pip install earthengine-api geemap rasterio shapely
import ee, geemap, glob, rasterio
from shapely.geometry import box, mapping

# 1) Authenticate Earth Engine
ee.Authenticate()
ee.Initialize(project='sensesaka2024')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.6 MB/s eta 0:00:00


GOAL IS TO CHECK ADDED VALUE OF DRONE-BASED CALIBRATION OF NBR AS KEY TO A ROBUST RBI

TRYING A DIFFERENT FIT FOR NBR-NDVI FIT

In [3]:
# 1) Install & import
!pip install ipyleaflet branca
from ipyleaflet import Map, DrawControl, basemaps, basemap_to_tiles
import json

# 2) Create a map centered on Santa Rosa (approx)
m = Map(
    layers=(basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),),
    center=(15.46, 121.07),
    zoom=12
)

# 3) Add a DrawControl for polygons
dc = DrawControl(
    marker={},  # disable markers
    circle={},  # disable circles
    circlemarker={},
    polyline={},
    polygon={
        "shapeOptions": {
            "fillColor": "#6beff0",
            "color": "#6beff0",
            "fillOpacity": 0.3
        }
    },
    rectangle={
        "shapeOptions": {
            "fillColor": "#fca45d",
            "color": "#fca45d",
            "fillOpacity": 0.3
        }
    },
)
m.add_control(dc)

# 4) Capture the last‐drawn feature as GeoJSON
aoi_geojson = None
def handle_draw(self, action, geo_json):
    global aoi_geojson
    if action == 'created':  # when user finishes drawing
        aoi_geojson = geo_json
        print("AOI captured!")
        # You can now remove the control if you like:
        m.remove_control(dc)

dc.on_draw(handle_draw)

# 5) Display the map and draw your AOI
m


Map(center=[15.46, 121.07], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

AOI captured!


In [42]:
m_coef = 0.9972435
b_coef = 0.04344955
date_start, date_end = '2025-03-01', '2025-04-30'
ptile = 75  # dynamic threshold (percentile)
key_name = f'RBI_p{ptile}'

# ---------------- Cloud/Shadow mask (no copyProperties here) -----------
def mask_s2_clouds(img):
    # S2 SR Harmonized has SCL (scene classification)
    scl = img.select('SCL')
    # 3=cloud shadow, 8=cloud (med), 9=cloud (high), 10=thin cirrus, 11=snow
    good = (scl.neq(3)
              .And(scl.neq(8))
              .And(scl.neq(9))
              .And(scl.neq(10))
              .And(scl.neq(11)))
    qa = img.select('QA60')
    cloudBitMask, cirrusBitMask = 1 << 10, 1 << 11
    qa_ok = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return img.updateMask(good).updateMask(qa_ok)

# WorldCover cropland (40) as rice proxy (static mask)
riceMask = (ee.Image('ESA/WorldCover/v100/2020').select('Map').eq(40))

# ---------------- Per-image RBI + dynamic threshold --------------------
def add_rbi_and_burned(img):
    im = mask_s2_clouds(img)

    ndvi = im.normalizedDifference(['B8','B4'])
    nbr  = im.normalizedDifference(['B8A','B12'])     # 20 m bands
    ndvi_cal = ndvi.multiply(m_coef).add(b_coef)
    rbi = nbr.subtract(ndvi_cal).divide(nbr.add(ndvi_cal)).rename('RBI')

    # restrict to cropland/rice
    rbi_rice = rbi.updateMask(riceMask)

    # dynamic threshold per image over AOI (20 m to match B8A/B12)
    stats = rbi_rice.reduceRegion(
        reducer   = ee.Reducer.percentile([ptile]),
        geometry  = aoi,
        scale     = 20,
        maxPixels = 1e9,
        bestEffort=True
    )

    # if no valid pixels, fall back to 0.2
    pxx = ee.Number(ee.Algorithms.If(
        ee.Dictionary(stats).contains(key_name),
        ee.Dictionary(stats).get(key_name),
        0.2
    ))

    burned = rbi_rice.gte(pxx).selfMask().rename('burned').uint8()

    # Attach properties at the end, then cast back to ee.Image to keep image methods
    out = rbi_rice.addBands(burned).set({
        'system:time_start': img.get('system:time_start'),
        key_name: pxx
    })
    return ee.Image(out)

# ---------------- Collections ----------------
s2_coll = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(aoi)
           .filterDate(date_start, date_end)
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 80))
           .map(lambda x: x.clip(aoi))
           .sort('system:time_start'))

s2_rgb_cm  = s2_coll.map(mask_s2_clouds)        # for visualization
rbi_series = s2_coll.map(add_rbi_and_burned)    # RBI + burned bands

# ---------------- Map ----------------
Map_ = geemap.Map(center=(15.46, 121.07), zoom=12)
Map_.add_basemap("HYBRID")

rgb_median = s2_rgb_cm.median()
rgbVis = {'bands':['B4','B3','B2'], 'min':0, 'max':3000, 'gamma':1.3}
Map_.addLayer(rgb_median, rgbVis, 'Median RGB', False)

rbiVis   = {'min':-0.5, 'max':0.5, 'palette':['blue','white','red']}
burnVis  = {'min':0, 'max':1, 'palette':['ffffff','ff0000']}  # white=bg, red=burned

size = rbi_series.size().getInfo()
rbi_list = rbi_series.toList(size)
rgb_list = s2_rgb_cm.toList(size)

for i in range(size):
    r_img  = ee.Image(rbi_list.get(i))
    s_img  = ee.Image(rgb_list.get(i))
    date_str = ee.Date(r_img.get('system:time_start')).format('YYYY-MM-dd').getInfo()
    pxx_val  = ee.Number(r_img.get(key_name)).getInfo()

    Map_.addLayer(s_img, rgbVis, f'RGB {date_str}', False)
    Map_.addLayer(r_img.select('RBI'), rbiVis, f'RBI {date_str}', False)
    Map_.addLayer(r_img.select('burned'), burnVis, f'Burned ≥ p{ptile} ({pxx_val:.3f}) {date_str}', False)

Map_

Map(center=[15.46, 121.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

In [36]:
# Make sure AOI exists
if aoi_geojson is None:
    raise ValueError("Draw AOI in the previous cell first.")

# 1) AOI geometry
aoi = ee.Geometry(aoi_geojson["geometry"])

# 2) Saved NDVI calibration parameters (drone → S2)
m_coef = 0.9972435
b_coef = 0.04344955

# 3) Cloud mask function using QA60
def mask_s2_qa60(image):
    """
    Mask clouds & cirrus in Sentinel-2 SR using QA60 band (bits 10 & 11).
    """
    qa = image.select("QA60")
    cloud_bit  = 1 << 10   # opaque clouds
    cirrus_bit = 1 << 11   # cirrus clouds

    mask = (qa.bitwiseAnd(cloud_bit).eq(0)
              .And(qa.bitwiseAnd(cirrus_bit).eq(0)))
    return image.updateMask(mask)

# 4) Load April 2025 Sentinel-2, cloud-mask, clip, median composite
s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(aoi)
      .filterDate("2025-03-01", "2025-04-30")
      .map(mask_s2_qa60)                               # 🟡 cloud mask here
      .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 80))
      .map(lambda img: img.clip(aoi))
      .median())

# 5) Compute NDVI and NBR
ndvi = s2.normalizedDifference(["B8", "B4"]).rename("NDVI")
nbr  = s2.normalizedDifference(["B8A", "B12"]).rename("NBR")

# 6) Apply calibration to NDVI
ndvi_cal = ndvi.multiply(m_coef).add(b_coef).rename("NDVI_cal")

# 7) Compute Rice Burned Index (RBI)
rbi = nbr.subtract(ndvi_cal).divide(nbr.add(ndvi_cal)).rename("RBI")

# 8) Mask RBI to ESA WorldCover rice class 40
riceMask = (ee.Image("ESA/WorldCover/v100/2020")
            .select("Map")
            .eq(40)
            .selfMask())
rbi_rice = rbi.updateMask(riceMask)

# 9) Map: RGB + RBI (rice only)
Map_ = geemap.Map(center=(15.46, 121.07), zoom=12)
Map_.add_basemap("HYBRID")

# Sentinel-2 RGB so you can visually inspect clouds & land
Map_.addLayer(
    s2,
    {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000, "gamma": 1.3},
    "S2 RGB (Apr 2025)"
)

# Calibrated RBI over rice paddies
Map_.addLayer(
    rbi_rice,
    {"min": -.1, "max": 0.9, "palette": ["blue", "white", "red"]},
    "Calibrated RBI (Rice)"
)


# Assuming 'rbi_rice' is your Calibrated RBI image as defined previously.

# 1. Define the classification thresholds
BURNED_MIN = -0.1
BURNED_MAX = 0.9

# 2. Create a mask where RBI is greater than or equal to MIN
# Result is 1 where True, 0 where False
min_mask = rbi_rice.gte(BURNED_MIN)

# 3. Create a mask where RBI is less than or equal to MAX
# Result is 1 where True, 0 where False
max_mask = rbi_rice.lte(BURNED_MAX)

# 4. Combine the masks using 'and' to select pixels WITHIN the range
# A pixel is classified as 'burned' (value 1) only if BOTH masks are True (1)
burned_mask = min_mask.And(max_mask)

# 5. Create the final classified image:
# Pixels in the mask (RBI between -0.3 and 0.9) get a value of 1 (BURNED),
# all others get a value of 0 (NOT BURNED).
burned_classification = burned_mask.rename("Burned_NotBurned")

# 6. Add the binary classification layer to the map
Map_.addLayer(
    burned_classification,
    {"min": 0, "max": 1, "palette": ["000000", "FF0000"]}, # Black for 0, Red for 1
    "Burned/Not Burned Classification"
)

Map_



Map(center=[15.46, 121.07], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataG…

## ENDS HERE........................OVERLAY THE BURNED PIXELS WITH DIGISAKA FARMS FOR ALERTS